In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
pd.set_option('display.max_columns', None)

# Procesado de los datos

## Lectura de datos

In [3]:
# Cargamos los datos y eliminamos las filas duplicadas. Un mismo vuelo no puede retrasarse en el mismo momento del tiempo más de una vez
df = pd.read_csv("../Data/flights.csv")
df = df.drop_duplicates()
df.head()

c:\Users\alvin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## Arreglo de datos

In [4]:
df['ORIGIN_AIRPORT'] = df['ORIGIN_AIRPORT'].astype(str)
df['DESTINATION_AIRPORT'] = df['DESTINATION_AIRPORT'].astype(str)

Hay una serie de códigos de aeropuertos de origen y destino que no corresponden con el IATA_CODE asociado al aeropuerto si no que aparece un id numérico.

In [5]:
print("Aeropuertos origen: "+ str(len(df[df['ORIGIN_AIRPORT'].str.isdigit()])))
print("Aeropuertos destino: "+ str(len(df[df['DESTINATION_AIRPORT'].str.isdigit()])))

Aeropuertos origen: 486165
Aeropuertos destino: 486165


Para corregir esto hacemos uso del archivo airports_dict, el cual genera un diccionario en el que se asocian las claves de aeropuerto numéricas a un código str como el que tenemos en el resto de casos en función de las rutas de los vuelos.

In [6]:
# Cargamos el diccionario
with open('dict_airport.json', 'rb') as fp:
    dict_airport = pickle.load(fp)

In [7]:
# Creamos dos columnas auxiliares para reemplazar el código numérico por el codigo str de aeropuerto correspondiente
df['CODE_ORI'] = df['ORIGIN_AIRPORT']
df['CODE_ORI'] = df['CODE_ORI'].map(dict_airport)

df['CODE_DEST'] = df['ORIGIN_AIRPORT']
df['CODE_DEST'] = df['DESTINATION_AIRPORT'].map(dict_airport)

# Sustituimos
df['CODE_ORI'] = df['CODE_ORI'].fillna(df['ORIGIN_AIRPORT'])
df['ORIGIN_AIRPORT'] = df['CODE_ORI']

df['CODE_DEST'] = df['CODE_DEST'].fillna(df['DESTINATION_AIRPORT'])
df['DESTINATION_AIRPORT'] = df['CODE_DEST']

df = df.drop(['CODE_ORI','CODE_DEST'],axis = 1)

In [8]:
# Comprobamos
print("Aeropuertos origen: "+ str(len(df[df['ORIGIN_AIRPORT'].str.isdigit()])))
print("Aeropuertos destino: "+ str(len(df[df['DESTINATION_AIRPORT'].str.isdigit()])))

Aeropuertos origen: 1235
Aeropuertos destino: 1235


**NOTA:** vemos que hay una serie de índices que el algoritmos no consigue emparejar con un aeropuerto. No obstante, estos datos no corresponden con una muestra representativa de los datos (poco volumen) por lo que los eliminamos

In [9]:
df = df.drop(df[df['ORIGIN_AIRPORT'].str.isdigit()].index)
df = df.drop(df[df['DESTINATION_AIRPORT'].str.isdigit()].index)

In [10]:
# Comprobamos parte II
print("Aeropuertos origen: "+ str(len(df[df['ORIGIN_AIRPORT'].str.isdigit()])))
print("Aeropuertos destino: "+ str(len(df[df['DESTINATION_AIRPORT'].str.isdigit()])))

Aeropuertos origen: 0
Aeropuertos destino: 0


## Análisis de variables

In [11]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5816609 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

Columnas que se pueden eliminar:
- TAIL NUMBER: representa un ID de avión único por lo que no corresponde con una variable representativa para el análisis por lo que la eliminamos

### Debatir:
Nos importan..?
- SCHEDULED_DEPARTURE: yo diría que no, si llega en hora = OK!
- DEPARTURE_TIME: es interesante analizar los retrasos por tramo horario? - SI
- DEPARTURE_DELAY: yo diría que no, si sale tarde pero llega en hora = OK! - SI
- ARRIVAL_TIME: yo diría que no, solo me interesa si el vuelo llega tarde, no? - SI
- SCHEDULED_TIME: no se a que se refiere  
- SCHEDULED_ARRIVAL: solo nos interesa saber si se retrasa o no, no la previsión
- TAXI_IN
- TAXI_OUT
- WHEELS_OFF
- WHEELS_ON
- AIR_TIME
- FLIGHT_NUMBER - SI
- DESTINATION_AIRPORT - DUDA

In [13]:
# Eliminamos tail number porque es un identificador de avión (no vuelo) único
df = df.drop("TAIL_NUMBER",axis=1)

In [14]:
# Eliminamos el resto de variables que no usaremos para nuestro análisis
df = df.drop(["SCHEDULED_DEPARTURE", "SCHEDULED_TIME", "SCHEDULED_ARRIVAL",
             "TAXI_IN", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "AIR_TIME"],axis=1)

Hay un error de formato con la variable flight number, que es un indicador del vuelo y por lo tanto una variable categórica. Lo mismo ocurre con Cancelled y Diverted

In [15]:
# Añadimos el FlightNum, Cancelled and Diverted como variables categóricas
df['FLIGHT_NUMBER']=df['FLIGHT_NUMBER'].astype(object) 
df['CANCELLED']=df['CANCELLED'].astype(object) 
df['DIVERTED']=df['DIVERTED'].astype(object) 

In [16]:
# Unimos las columnas Year, Month y Day of Month como una única variable fecha
# Formato por defecto mes/dia/año para que to_datetime funcione correctamente
df["DATE"]  = df['MONTH'].astype(str) +'/'+ df['DAY'].astype(str) +'/' + df['YEAR'].astype(str)
df["DATE"] = pd.to_datetime(df["DATE"])

Si nos interesa en algún momento podríamos añadir las horas también 

In [17]:
#Eliminamos las columnas year, month y day
df = df.drop(["YEAR","MONTH", "DAY"],axis=1)

# También podemos eliminar la columna DAY_OF_WEEK ya que podemos obtenerla
df = df.drop("DAY_OF_WEEK",axis=1)
print(f"Ejemplo: obtener dia de la semana de la fecha {df['DATE'].iloc[0]} --> {df['DATE'].iloc[0].day_name()} (dia {df['DATE'].iloc[0].dayofweek})")

# NOTA: hay que tener en cuenta que dayofweek empieza a contar en 0 = lunes

Ejemplo: obtener dia de la semana de la fecha 2015-01-01 00:00:00 --> Thursday (dia 3)


In [18]:
# Cambiamos el orden de las columnas, para que DATE sea la primera
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols] 
df.head()

,DATE,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015-01-01,AS,98,ANC,SEA,2354.0,-11.0,194.0,1448,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,AA,2336,LAX,PBI,2.0,-8.0,279.0,2330,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,US,840,SFO,CLT,18.0,-2.0,293.0,2296,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01,AA,258,LAX,MIA,15.0,-5.0,281.0,2342,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,AS,135,SEA,ANC,24.0,-1.0,215.0,1448,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


Hablar formato tiempos: 
    SCHEDULED_DEPARTURE, DEPARTURE_TIME, SCHEDULED_TIME
    SCHEDULED_ARRIVAL, ARRIVAL_TIME (se podrían eliminar dado que nos sirven para sacar arrival delay)

## Debatir:

Las variables SCHEDULED_DEPARTURE, DEPARTURE_TIME, DEPARTURE_DELAY, ARRIVAL_TIME y SCHEDULED_ARRIVAL nos ayudan a saber si un vuelo se ha retrasado o no, pero una vez aue sabemos esto no tienen  valor en sí mismas por lo que las podemos eliminar

#### Tipos de vuelo:
1. On time/ arrived earlier --> arrival_delay <=0
2. Delayed   --> arrival_delay > 0
3. Diverted  --> diverted == 1
4. Cancelled --> cancelled == 1

##### Razones por las que se retrasa un vuelo:
- AIR_SYSTEM_DELAY     
- SECURITY_DELAY       
- AIRLINE_DELAY        
- LATE_AIRCRAFT_DELAY 
- WEATHER_DELAY       

In [19]:
# Vamos a examinar ahora los valores nulos
df.isna().sum()

DATE                         0
AIRLINE                      0
FLIGHT_NUMBER                0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
DEPARTURE_TIME           86129
DEPARTURE_DELAY          86129
ELAPSED_TIME            105040
DISTANCE                     0
ARRIVAL_TIME             92485
ARRIVAL_DELAY           105040
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5726751
AIR_SYSTEM_DELAY       4753413
SECURITY_DELAY         4753413
AIRLINE_DELAY          4753413
LATE_AIRCRAFT_DELAY    4753413
WEATHER_DELAY          4753413
dtype: int64

Las variables CANCELLATION_REASON, AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY,WEATHER_DELAY presentan una gran cantidad de valores nulos. No obstante, todas estas variables se relacionan con vuelos cancelados o retrasados por lo que tiene sentido que sean valores nulos para aquellos vuelos que no se hayan cancelado ni retrasado. Vamos a analizar estas variables

In [20]:
cancelled = df[df['CANCELLED'] == 1]
cancelled.isna().sum()

DATE                       0
AIRLINE                    0
FLIGHT_NUMBER              0
ORIGIN_AIRPORT             0
DESTINATION_AIRPORT        0
DEPARTURE_TIME         86129
DEPARTURE_DELAY        86129
ELAPSED_TIME           89858
DISTANCE                   0
ARRIVAL_TIME           89858
ARRIVAL_DELAY          89858
DIVERTED                   0
CANCELLED                  0
CANCELLATION_REASON        0
AIR_SYSTEM_DELAY       89858
SECURITY_DELAY         89858
AIRLINE_DELAY          89858
LATE_AIRCRAFT_DELAY    89858
WEATHER_DELAY          89858
dtype: int64

Vemos como en este caso no existen valores nulos para la columna CANCELLATION_REASON. No obstante, nuestro análisis consiste en prededcir el retraso de vuelos por lo que no necesitamos la info de vuelos cancelados/ redirigidos así que eliminamos dichos registros

In [21]:
# Eliminamos los cancelados
df = df[df['CANCELLED'] == 0]

#Eliminamos los redirigidos
df = df[df['DIVERTED'] == 0]

df = df.drop(["DIVERTED","CANCELLED", "CANCELLATION_REASON"],axis=1)

Nos quedamos solo con los vuelos que llegan antes de lo previsto, en hora o con retraso

In [22]:
len(df)

5711569

In [23]:
df.isna().sum()

DATE                         0
AIRLINE                      0
FLIGHT_NUMBER                0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
DEPARTURE_TIME               0
DEPARTURE_DELAY              0
ELAPSED_TIME                 0
DISTANCE                     0
ARRIVAL_TIME                 0
ARRIVAL_DELAY                0
AIR_SYSTEM_DELAY       4648373
SECURITY_DELAY         4648373
AIRLINE_DELAY          4648373
LATE_AIRCRAFT_DELAY    4648373
WEATHER_DELAY          4648373
dtype: int64

Realizamos ahora el análisis de los valores nulos relacionados con los vuelos retrasados.

**NOTA**: consideramos que un vuelo se retrasa si llega pasada la hora prevista, independientemente de si ha tenido retraso en la hora de salida o no

In [24]:
delayed = df[df['ARRIVAL_DELAY'] > 0]
delayed.isna().sum()

DATE                         0
AIRLINE                      0
FLIGHT_NUMBER                0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
DEPARTURE_TIME               0
DEPARTURE_DELAY              0
ELAPSED_TIME                 0
DISTANCE                     0
ARRIVAL_TIME                 0
ARRIVAL_DELAY                0
AIR_SYSTEM_DELAY       1023002
SECURITY_DELAY         1023002
AIRLINE_DELAY          1023002
LATE_AIRCRAFT_DELAY    1023002
WEATHER_DELAY          1023002
dtype: int64

Si considerabamos todos los vuelos teníamos 1283118 registros con valor nulo en las variables DELAY, cuando filtramos por vuelos retrasados tenemos 275025.
Vamos a analizar primero los valores nulos de aquellos vuelos que se han retrasado

In [25]:
delayed = delayed[['AIR_SYSTEM_DELAY', 'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']]
delayed.head()

,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
2,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN


In [26]:
# Seleccionamos todas las filas que tengan NaN en todas las columnas
nulls = delayed.loc[(delayed['AIR_SYSTEM_DELAY'].isnull() == True) & (delayed['SECURITY_DELAY'].isnull() == True) & (delayed['AIRLINE_DELAY'].isnull() == True) & (delayed['LATE_AIRCRAFT_DELAY'].isnull() == True) & (delayed['WEATHER_DELAY'].isnull() == True)]
len(nulls)

1023002

Vemos que TODOS los NaN se concentran en las mismas filas. Entendemos que en este caso el vuelo se ha retrasado por causa desconocida. Para indicar esto creamos una nueva columna 'OTHER_DELAY' en nuestro data frame cuyo valor sea igual al delay

In [27]:
# Creamos la OTHER_DELAY con los mismos datos que ARRIVAL_DELAY
df['OTHER_DELAY'] = df['ARRIVAL_DELAY']

# Como hemos visto que si una columna _DELAY es NaN el resto también, utilizamos una única columa para comparar, y asignamos a OTHER_DELAY la diferencia entre el delay a la llegada y el resto de delays
df.loc[pd.notna(df['AIR_SYSTEM_DELAY']),'OTHER_DELAY'] = df['ARRIVAL_DELAY'] - df['AIR_SYSTEM_DELAY'] - df['SECURITY_DELAY'] - df['AIRLINE_DELAY'] -df['LATE_AIRCRAFT_DELAY'] - df['WEATHER_DELAY']
df[45:50]

,DATE,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY
47,2015-01-01,AS,114,ANC,SEA,209.0,-11.0,199.0,1448,628.0,-12.0,NaN,NaN,NaN,NaN,NaN,-12.0
48,2015-01-01,B6,668,PSE,MCO,248.0,-7.0,183.0,1179,451.0,-9.0,NaN,NaN,NaN,NaN,NaN,-9.0
49,2015-01-01,UA,1162,BQN,EWR,258.0,-1.0,247.0,1585,605.0,6.0,NaN,NaN,NaN,NaN,NaN,6.0
50,2015-01-01,B6,1030,BQN,MCO,304.0,-3.0,196.0,1129,520.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0
51,2015-01-01,B6,262,SJU,BOS,316.0,-14.0,243.0,1674,619.0,-16.0,NaN,NaN,NaN,NaN,NaN,-16.0


In [28]:
# Cambiamos OTHEY_DELAY <0 por 0 dado que estos vuelos han llegado antes de lo previsto, no han experimentado un retraso
df.loc[df["OTHER_DELAY"] < 0, "OTHER_DELAY"] = 0
df[45:50]

,DATE,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY
47,2015-01-01,AS,114,ANC,SEA,209.0,-11.0,199.0,1448,628.0,-12.0,NaN,NaN,NaN,NaN,NaN,0.0
48,2015-01-01,B6,668,PSE,MCO,248.0,-7.0,183.0,1179,451.0,-9.0,NaN,NaN,NaN,NaN,NaN,0.0
49,2015-01-01,UA,1162,BQN,EWR,258.0,-1.0,247.0,1585,605.0,6.0,NaN,NaN,NaN,NaN,NaN,6.0
50,2015-01-01,B6,1030,BQN,MCO,304.0,-3.0,196.0,1129,520.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0
51,2015-01-01,B6,262,SJU,BOS,316.0,-14.0,243.0,1674,619.0,-16.0,NaN,NaN,NaN,NaN,NaN,0.0


In [29]:
# Early arrival flights
early_arrival = df[df['ARRIVAL_DELAY']<=0]

print(len(early_arrival))
early_arrival.isna().sum()

3625371


DATE                         0
AIRLINE                      0
FLIGHT_NUMBER                0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
DEPARTURE_TIME               0
DEPARTURE_DELAY              0
ELAPSED_TIME                 0
DISTANCE                     0
ARRIVAL_TIME                 0
ARRIVAL_DELAY                0
AIR_SYSTEM_DELAY       3625371
SECURITY_DELAY         3625371
AIRLINE_DELAY          3625371
LATE_AIRCRAFT_DELAY    3625371
WEATHER_DELAY          3625371
OTHER_DELAY                  0
dtype: int64

El resto de valores NaN en dichas columnas corresponden a aquellos vuelos que han llegado antes de tiempo a destino, cosa que tiene sentido dado que no han experimentado ningún delay

Una vez analizado el por qué de los valores NaN presentes en el dataset parece razonable sustituir dichos valores por 0

In [30]:
# Asignamos a todos los NaNs el valor 0, ya que ahora todo el retraso de sus vuelos está plasmado en la variable OTHER_DELAY
df = df.fillna(0)
df[45:50]

,DATE,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY
47,2015-01-01,AS,114,ANC,SEA,209.0,-11.0,199.0,1448,628.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
48,2015-01-01,B6,668,PSE,MCO,248.0,-7.0,183.0,1179,451.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
49,2015-01-01,UA,1162,BQN,EWR,258.0,-1.0,247.0,1585,605.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0
50,2015-01-01,B6,1030,BQN,MCO,304.0,-3.0,196.0,1129,520.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0
51,2015-01-01,B6,262,SJU,BOS,316.0,-14.0,243.0,1674,619.0,-16.0,0.0,0.0,0.0,0.0,0.0,0.0


fillna() sustituye los NaN de todo el dataframe por lo que es importante destacar que podemos utilizar este método dado que los únicos campos NaN del dataframe se encuentran en las variables de delay analizadas. 

In [31]:
df.isna().sum()

DATE                   0
AIRLINE                0
FLIGHT_NUMBER          0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
DEPARTURE_TIME         0
DEPARTURE_DELAY        0
ELAPSED_TIME           0
DISTANCE               0
ARRIVAL_TIME           0
ARRIVAL_DELAY          0
AIR_SYSTEM_DELAY       0
SECURITY_DELAY         0
AIRLINE_DELAY          0
LATE_AIRCRAFT_DELAY    0
WEATHER_DELAY          0
OTHER_DELAY            0
dtype: int64

Resulta interesante saber si los vuelos pueden retrasarse por un único motivo o exclusivamente por uno. Vamos a investigarlo

In [32]:
# Tomamos la variable WEATHER_DELAY como referencia
weather = df.loc[(df['WEATHER_DELAY'] != df['ARRIVAL_DELAY']) & (df['WEATHER_DELAY']>0 )]
weather.head()

,DATE,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY
165,2015-01-01,EV,5976,BUF,EWR,928.0,213.0,93.0,282,1101.0,226.0,13.0,0.0,0.0,0.0,213.0,0.0
174,2015-01-01,UA,247,PHX,IAH,751.0,111.0,160.0,1009,1131.0,123.0,12.0,0.0,0.0,0.0,111.0,0.0
344,2015-01-01,OO,5466,CLD,LAX,654.0,54.0,85.0,86,819.0,88.0,34.0,0.0,0.0,0.0,54.0,0.0
369,2015-01-01,OO,6420,PSP,DEN,812.0,131.0,144.0,776,1136.0,144.0,13.0,0.0,0.0,0.0,131.0,0.0
502,2015-01-01,OO,6571,DRO,PHX,631.0,16.0,124.0,351,835.0,56.0,40.0,0.0,0.0,0.0,16.0,0.0


Confirmamos, el retraso puedes estar asociado a varios motivos

## CHECKPOINT. Datos de vuelos limpios

In [33]:
df.head()

,DATE,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY
0,2015-01-01,AS,98,ANC,SEA,2354.0,-11.0,194.0,1448,408.0,-22.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-01,AA,2336,LAX,PBI,2.0,-8.0,279.0,2330,741.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-01,US,840,SFO,CLT,18.0,-2.0,293.0,2296,811.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0
3,2015-01-01,AA,258,LAX,MIA,15.0,-5.0,281.0,2342,756.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-01,AS,135,SEA,ANC,24.0,-1.0,215.0,1448,259.0,-21.0,0.0,0.0,0.0,0.0,0.0,0.0


## Info Aerolíneas

In [34]:
# Vamos a añadir la el nombre asociado a las airlines
# Cargamos los datos
airlines = pd.read_csv("../Data/airlines.csv")
airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [35]:
# Vamos a examinar ahora los valores nulos
airlines.isna().sum()

IATA_CODE    0
AIRLINE      0
dtype: int64

In [36]:
# Renombramos la columna "AIRLINE" para poder hacer el join con la tabla de aerolineas
airlines = airlines.rename(columns={"IATA_CODE": "AIRLINE_CODE"})
df = df.rename(columns={"AIRLINE": "AIRLINE_CODE"})
df.head()

,DATE,AIRLINE_CODE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY
0,2015-01-01,AS,98,ANC,SEA,2354.0,-11.0,194.0,1448,408.0,-22.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-01,AA,2336,LAX,PBI,2.0,-8.0,279.0,2330,741.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-01,US,840,SFO,CLT,18.0,-2.0,293.0,2296,811.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0
3,2015-01-01,AA,258,LAX,MIA,15.0,-5.0,281.0,2342,756.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-01,AS,135,SEA,ANC,24.0,-1.0,215.0,1448,259.0,-21.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Unimos ambas tablas
flights = df.merge(airlines, on='AIRLINE_CODE', how='left')

In [38]:
# Ponemos la nueva columna a continuación del código de la aerolinea
cols = flights.columns.tolist()
cols = cols[0:2]+cols[-1:] + cols[2:-1]
flights = flights[cols] 
flights.head()

,DATE,AIRLINE_CODE,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY
0,2015-01-01,AS,Alaska Airlines Inc.,98,ANC,SEA,2354.0,-11.0,194.0,1448,408.0,-22.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-01,AA,American Airlines Inc.,2336,LAX,PBI,2.0,-8.0,279.0,2330,741.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-01,US,US Airways Inc.,840,SFO,CLT,18.0,-2.0,293.0,2296,811.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0
3,2015-01-01,AA,American Airlines Inc.,258,LAX,MIA,15.0,-5.0,281.0,2342,756.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-01,AS,Alaska Airlines Inc.,135,SEA,ANC,24.0,-1.0,215.0,1448,259.0,-21.0,0.0,0.0,0.0,0.0,0.0,0.0


## Info Aeropuertos

In [44]:
# Vamos a añadir la el nombre asociado a las airlines
# Cargamos los datos
airports = pd.read_csv("../Data/airports.csv")
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [45]:
# Vamos a examinar ahora los valores nulos
airports.isna().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     3
LONGITUDE    3
dtype: int64

Falta información de 3 aeropuertos, así que la rellenaremos buscando sus datos en internet e introduciéndola manualmente

In [46]:
ECP_COORD = [30.3549, 85.7995]
PBG_COORD = [44.6521, 73.4679]
UST_COORD = [29.9544, 81.3429]
airports.at[96,["LATITUDE","LONGITUDE"]]= ECP_COORD
airports.at[234,["LATITUDE","LONGITUDE"]]= PBG_COORD
airports.at[313,["LATITUDE","LONGITUDE"]]= UST_COORD
airports.isna().sum()

IATA_CODE    0
AIRPORT      0
CITY         0
STATE        0
COUNTRY      0
LATITUDE     0
LONGITUDE    0
dtype: int64

Eliminamos la columna COUNTRY, ya que todos los aerupuertos son de Estados Unidos

In [50]:
airports = airports.drop("COUNTRY", axis=1)

Cambiamos el nombre de la columna airport, ya que trabajaremos únicamente con los códigos de los aeropuertos y con sus nombres

In [51]:
airports= airports.rename(columns={"AIRPORT": "AIRPORT_NAME"})
airports= airports.rename(columns={"IATA_CODE": "AIRPORT"})

In [52]:
# Creamos dos bases de datos para hacer el join con la principal tanto para aeropuertos de llegada como de salida
airports= airports.rename(columns={"IATA_CODE": "AIRPORT"})

origin_airports = airports.add_prefix('ORIGIN_')
destination_airports = airports.add_prefix('DESTINATION_')

origin_airports.head()

,ORIGIN_AIRPORT,ORIGIN_AIRPORT_NAME,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_LATITUDE,ORIGIN_LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,31.53552,-84.19447


In [54]:
# Unimos las tres tablas
flights_origin = flights.merge(origin_airports, on='ORIGIN_AIRPORT', how='left')
flights_complete = flights_origin.merge(destination_airports, on='DESTINATION_AIRPORT', how='left')
flights_complete.head()

,DATE,AIRLINE_CODE,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,ELAPSED_TIME,DISTANCE,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,OTHER_DELAY,ORIGIN_AIRPORT_NAME,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DESTINATION_AIRPORT_NAME,DESTINATION_CITY,DESTINATION_STATE,DESTINATION_LATITUDE,DESTINATION_LONGITUDE
0,2015-01-01,AS,Alaska Airlines Inc.,98,ANC,SEA,2354.0,-11.0,194.0,1448,408.0,-22.0,0.0,0.0,0.0,0.0,0.0,0.0,Ted Stevens Anchorage International Airport,Anchorage,AK,61.17432,-149.99619,Seattle-Tacoma International Airport,Seattle,WA,47.44898,-122.30931
1,2015-01-01,AA,American Airlines Inc.,2336,LAX,PBI,2.0,-8.0,279.0,2330,741.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,Los Angeles International Airport,Los Angeles,CA,33.94254,-118.40807,Palm Beach International Airport,West Palm Beach,FL,26.68316,-80.09559
2,2015-01-01,US,US Airways Inc.,840,SFO,CLT,18.0,-2.0,293.0,2296,811.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,San Francisco International Airport,San Francisco,CA,37.61900,-122.37484,Charlotte Douglas International Airport,Charlotte,NC,35.21401,-80.94313
3,2015-01-01,AA,American Airlines Inc.,258,LAX,MIA,15.0,-5.0,281.0,2342,756.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,Los Angeles International Airport,Los Angeles,CA,33.94254,-118.40807,Miami International Airport,Miami,FL,25.79325,-80.29056
4,2015-01-01,AS,Alaska Airlines Inc.,135,SEA,ANC,24.0,-1.0,215.0,1448,259.0,-21.0,0.0,0.0,0.0,0.0,0.0,0.0,Seattle-Tacoma International Airport,Seattle,WA,47.44898,-122.30931,Ted Stevens Anchorage International Airport,Anchorage,AK,61.17432,-149.99619


In [55]:
len(flights_complete)

5711569

In [57]:
# Guardamos los datos preprocesados, para ser utilizados en la predicción posterior
flights_complete.to_parquet("flightsCleaned.parquet", index=False)